In [ ]:
#!pip install langchain openai fastapi uvicorn nest-asyncio

import nest_asyncio
import os
from fastapi import FastAPI, Request
import uvicorn
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate
import threading

# Enable nested event loops (needed in Colab)
nest_asyncio.apply()

app = FastAPI()

# Initialize OpenAI LLM
from langchain import OpenAI

llm = OpenAI(api_key='sk-proj-_qfi5UU1xQGFNyNhmQfR2BQQi_xyzGEik666dkUuBkOOw3p9LqJe3SK99S-Fg78RkGSPkttT3BlbkFJ1PUbI_pF8w0-CXFmzVBlxxHnVJrvNbJVFqBEw6iLN7bX3VPM6eMhjLKqXEsAsMUwtTSjjsxRAA')

# Prompts
summary_prompt = PromptTemplate(template="Summarize the following text:\n{text}")
keyword_prompt = PromptTemplate(template="Extract keywords from the following text:\n{text}")
response_prompt = PromptTemplate(
    template="Generate a response based on the summary '{summary}' and keywords '{keywords}':"
)

# Chains with invoke()
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
keyword_chain = LLMChain(llm=llm, prompt=keyword_prompt)
response_chain = LLMChain(llm=llm, prompt=response_prompt)

def execute_workflow(input_text):
    try:
        print("Generating summary...")
        summary = summary_chain.invoke(text=input_text)
        print("Summary:", summary)
        print("Extracting keywords...")
        keywords = keyword_chain.invoke(text=input_text)
        print("Keywords:", keywords)
        print("Generating response...")
        response = response_chain.invoke(summary=summary, keywords=keywords)
        print("Response:", response)
        return {
            "summary": summary,
            "keywords": keywords,
            "response": response
        }
    except Exception as e:
        print("Error during workflow:", e)
        return {"error": str(e)}

@app.post("/process")
async def process(request: Request):
    try:
        data = await request.json()
    except Exception as e:
        return {"error": f"Invalid JSON: {e}"}
    input_text = data.get("text")
    if not input_text:
        return {"error": "Missing 'text' in request body."}
    result = execute_workflow(input_text)
    return result

# Run server on port 8001
def run():
    uvicorn.run(app, host="0.0.0.0", port=8001)

# Start server in background thread
threading.Thread(target=run, daemon=True).start()

In [ ]:
import requests
import time

# Wait for server to start up
time.sleep(3)

payload = {
    "text": "Google Colab allows running Python code interactively in the cloud."
}
response = requests.post("http://127.0.0.1:8001/process", json=payload)

print("Status code:", response.status_code)
print("Response body:", response.text)

try:
    print("JSON output:", response.json())
except Exception as e:
    print("Error decoding JSON:", e)